In [1]:
# default_exp core

# to csv

> API details.

In [1]:
#hide
from nbdev.showdoc import *

In [7]:
import stravalib
from http.server import HTTPServer 
import urllib.parse
import webbrowser
import pandas as pd
import datetime

/home/tako/dev/strava_overview/env/lib/python3.7/site-packages/pandas/compat/__init__.py:117: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


In [8]:

class MyHandler(BaseHTTPServer.BaseHTTPRequestHandler):
  #Handle the web data sent from the strava API

  def do_HEAD(self):
    return self.do_GET()

  def do_GET(self):
    #Get the API code for Strava
    self.wfile.write('<script>window.close();</script>')
    code = parse.parse_qs(urlparse.urlparse(self.path).query)['code'][0]

    #Login to the API
    client  = UseCode(code)

    #Retrieve the last limit activities
    activities = GetActivities(client,limit)

    #Loop through the activities, and create a dict of the dataframe stream data of each activity
    print "looping through activities..."
    df_lst = {}
    for act in activities:
        print act.name
        df_lst[act.start_date] = ParseActivity(act,types)

    #create the concatenated df and fill null values
    df_total = concatdf(df_lst)
    df_total.fillna('',inplace=True)

    #Calculate the timestamp column
    df_total = df_total.reset_index(level=0)
    df_total['timestamp'] = pd.to_datetime(map(calctime, df_total['time'], df_total['level_0'])).to_pydatetime()

    #Split out lat and long columns
    df_total['lat'] = map(split_lat, (df_total['latlng']))
    df_total['long'] = map(split_long, (df_total['latlng']))

    #Index by startdate and timestamp, and drop arbitrary columns
    df_total = df_total.set_index(['act_startDate','timestamp'])
    df_total.drop(['latlng', 'level_0'], axis=1, inplace=True)

    #Write the file to a CSV - this will end up in your working directory
    now = datetime.datetime.now()
    df_total.to_csv('RideData_' + str(now.strftime('%Y%m%d%H%M%S')) + '.csv')
    print 'script complete!'


SyntaxError: Missing parentheses in call to 'print'. Did you mean print("looping through activities...")? (<ipython-input-8-909aa50dbd40>, line 19)